<a href="https://colab.research.google.com/github/opf-ute/scripts/blob/master/acopf5barras_2905.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Se quiere modelar y resolver el problema de flujo óptimo para el sistema representado por el unifilar de la figura. Se utilizará el modelo AC-OPF (BIM)



In [0]:
from IPython.display import Image
from IPython.core.display import HTML 

Image(url= "https://drive.google.com/uc?id=1zKqS2zehsZ2IxdM8sdNr7CHbdVfmTxtW", width=750)




In [0]:
!pip install cvxpy
!pip install git+https://github.com/cvxgrp/cvxpy.git

  Cloning https://github.com/cvxgrp/cvxpy.git to /tmp/pip-req-build-9d498kkh
  Running command git clone -q https://github.com/cvxgrp/cvxpy.git /tmp/pip-req-build-9d498kkh
  Stored in directory: /tmp/pip-ephem-wheel-cache-musy96d0/wheels/0e/7c/42/bc3294fba335fafe64e2ae05be0ce1b671bbe20244056cecae
Successfully built cvxpy
  Found existing installation: cvxpy 1.0.15
    Uninstalling cvxpy-1.0.15:
      Successfully uninstalled cvxpy-1.0.15


Bus Injection Model (BIM):

$$\mathbf{I}=\mathbf{Y}\mathbf{V}$$


$$\begin{bmatrix}
I_1\\
I_2\\
\vdots\\
I_N\end{bmatrix} = \begin{bmatrix}\ddots&&\\&Y_{kk}&Y_{ki}\\&&\ddots\end{bmatrix}\begin{bmatrix}
V_1\\
V_2\\
\vdots\\
V_N\end{bmatrix}$$


Donde $\mathbf{Y}$ es la matriz de admitancias con entradas

$$\left\{\begin{array}{lll}
Y_{kk}=&y_k+\sum_{i\sim k}y_{ki}&\\
Y_{ki}=&-y_{ki}&\text{si } k\sim i\\
Y_{ki}=&0&\text{en otro caso}
\end{array}\right.$$


Sea $Y_k$ la matriz que solamente contiene la k-esima fila de $Y$. Puede escribirse como

$$Y_k = E_k Y$$

Donde $E_k$ es cero salvo en la entrada $kk$, donde es $1$.

Se cumple
 
 $$Y^*_k=\left(\frac{Y^*_k+Y_k}{2}\right)+j\left(\frac{Y^*_k-Y_k}{2j}\right)=\Phi_k+j\Psi_k$$
 
 La potencia aparente inyectada en cada barra resulta
 
 
 $$\boxed{s_k = V\Phi_k V^*+jV^*\Psi_k V}$$

<hr>
 
 
Matriz de incidencia:

$$
A=\begin{bmatrix}
    0  & 0 & 0 & -1 & 1 \\
   -1  & 0 & 0 & 0 & 1 \\
   1  & 0 & 0 & -1 & 0 \\
   0  & 0 & -1 &1 & 0 \\
   1  & -1 & 0 & 0 & 0 \\
   0  & -1 & 1 & 0 & 0 \\
\end{bmatrix}
$$

Reactancias de linea:

$$
X=\begin{bmatrix}
    x_{54}  & 0 & 0 & 0 & 0 & 0\\
   0  & x_{51} & 0 & 0 & 0 & 0\\
   0  & 0 & x_{14} &  0 & 0 & 0 \\
   0  & 0 & 0 & x_{43} & 0 & 0 \\
   0  & 0 & 0 & 0 & x_{12} & 0 \\
   0  & 0 & 0 & 0 & 0 & x_{32} \\
\end{bmatrix}
$$

con $x_{12}=0.0281$, $x_{23}=0.0108$, $x_{34}=0.0297$, $x_{45}=0.0297$, $x_{14}=0.0304$, $x_{15}=0.0064$ (en p.u.).





In [0]:
Image(url= "https://drive.google.com/uc?id=1uoDeKu9d-sZhFM3R-2gmdplqNirh0cFf", width=750)


Luego el flujo de carga óptimo para esta red puede obtenerse con el siguiente codigo en CVXPY


In [0]:
# Import packages.
import cvxpy as cp
import numpy as np



In [15]:
# Modelo de cinco barras a modificar
Sbase = 100 # MVA
M=6
N=5
# Matriz de incidencia
A=np.array([[0, 0, 0, -1, 1],
            [-1, 0, 0, 0, 1],
            [1, 0, 0, -1, 0],
            [0, 0, -1, 1, 0],
            [1, -1, 0, 0, 0],
            [0, -1, 1, 0, 0] ]) 

# Reactancias en pu
x54 = 0.0297;
x51 = 0.0064;
x14 = 0.0304;
x43 = 0.0297;
x12 = 0.0281;
x32 = 0.0108;

X=np.array([[x54,0,0,0,0,0],
               [0,x51,0,0,0,0],
               [0,0,x14,0,0,0],
               [0,0,0,x43,0,0],
               [0,0,0,0,x12,0],
               [0,0,0,0,0,x32]])

R=X/100
Z=R+1j*X

#X=1j*np.array([[x54,0,0,0,0,0],
 #              [0,x51,0,0,0,0],
  #             [0,0,x14,0,0,0],
   #            [0,0,0,x43,0,0],
    #           [0,0,0,0,x12,0],
     #          [0,0,0,0,0,x32]])


#H=np.linalg.inv(X)@A
H=np.linalg.inv(Z)@A

Y = A.transpose()@H
Y = np.array(Y)


# ----------------------------
# Caracteristicas del problema
# ----------------------------
d = np.array([0,300,300,400,0]) / Sbase # Demanda
plmax = np.array([240,400,400,400,400,400]) / Sbase # Limite lineas
gmax = np.array([40,170,520,200,600]) / Sbase # Limite generacion
c = np.array([14,15,30,40,10]) # Costo

# G2pg pasa de generadores a generacion por barra:
# pg = G2pg * g
G2pg=np.array([[1,1,0,0,0],
               [0,0,0,0,0],
               [0,0,1,0,0],
               [0,0,0,1,0],
               [0,0,0,0,1]])



# --------------------------
# Crear matrices hermiticas
# --------------------------

# Asociadas al balance de cada nodo
# ---------------------------------
E = np.zeros([5,5,5])
y = np.zeros([5,5,5], dtype=np.complex64)
Phi = np.zeros([5,5,5], dtype=np.complex64)
Psi = np.zeros([5,5,5], dtype=np.complex64)
                                  

for k in range(0,5):
  E[k,k,k] = 1 # Todo ceros salvo en el kk-esimo elemento
  y[k] = E[k] @ Y # y[k] tiene en su k-esima fila la k-esima fila de Y
  Phi[k] = .5 * (np.matrix(y[k]) + np.matrix(y[k]).H) 
  Psi[k] = .5 /(1j) * (-np.matrix(y[k]) + np.matrix(y[k]).H)



# ---------------------------------------  
# Asociadas a las restricciones en lineas
# ---------------------------------------
D = [(4,3), (4,0), (0,3), (3,2), (0,1), (2,1)] #Linea 1 conecta (4->3); Linea 2 conecta (4->0), etc.

ek = [[0,0,0,0,1], # linea 1 sale de barra 5
      [0,0,0,0,1], # linea 2 sale de barra 5
      [1,0,0,0,0], # Linea 3 sale de barra 1 ...
      [0,0,0,1,0],
      [1,0,0,0,0],
      [0,0,1,0,0]]

# Matrices de restricciones en las lineas
M = np.zeros([6,5,5], dtype=np.complex64)                 
N = np.zeros([6,5,5], dtype=np.complex64)

for k in range(0,6):
  Yki = np.conj(Y[D[k]]) * np.outer(A[k], ek[k])  
  M[k] = .5 * (np.matrix(Yki) + np.matrix(Yki).H)  
  N[k] = .5 * (-np.matrix(Yki) + np.matrix(Yki).H)  



                 
#for k in range(0,5):
#  print('Phi[',k,'] es hermitica : ', np.allclose(np.matrix(Phi[k]),np.matrix(Phi[k]).H))

# C0 = c[0]*Phi[0] + c[1]*Phi[1] + c[2]*Phi[2] + c[3]*Phi[3] + c[4]*Phi[4]
                               

  
  
  
# ------------------
# Variables de CVXPY
# ------------------
pb = cp.Variable(5) # potencia neta inyectada en las barras = pg-d
g = cp.Variable(5) # potencia de cada generador 
pl = cp.Variable(6) # potencia por las líneas
pg = cp.Variable(5) # potencias inyectadas por los generadores en las barras; pg = G2pg*g

W = cp.Variable((5,5), hermitian=True) # Matriz hermitica, (con suerte) Rank=1
X = cp.Variable((5,5))
Y = cp.Variable((5,5))

#cost = cp.trace(A*X - B*Y)
cost = (c*g)

constraints = [G2pg*g == pg, pb==pg-d, g>=0 , g<=gmax,  W>>0]#, pl <= plmax, pl >= -plmax]
constraints.append(X == cp.real(W))
constraints.append(Y == cp.imag(W))


# Balance por nodo
for k in range(0,5):
  Phir = np.real(Phi[k])
  Phii = np.imag(Phi[k])
  constraints.append(cp.trace(Phir*X - Phii*Y) == pb[k])
  constraints.append(X[k,k]<=1.1)
# Restricciones de linea
#for k in range(0,6):
  #Mr = np.real(M[k])
  #Mi = np.imag(M[k])
  #constraints.append(cp.trace(Mr*X - Mi*Y) == -pl[k])
  
#constraints.append(cp.diag(X)<=1.1)
#constraints.append(cp.diag(X)>=.9)

prob = cp.Problem(cp.Minimize(cost),constraints)

prob.solve(verbose=True) #Poniendo verbose=False se oculta el Solver




# ------------------
# MOSTRAR RESULTADOS
# ------------------

print()
print()
print('RESULTADOS:')
print("El costo óptimo es", prob.value*100)
print()
print('La generacion optima es', g.value*100)
print('La generacion por barra es', pg.value*100)
print('pb es', pb.value*100)
print("En cada barra, se inyecta", pb.value*100)
#print('El flujo de activa por las lineas es:', pl.value*100)


print()
print()
#print('Multiplicadores de Lagrange:')
#for k in range(0,7):
#  print(constraints[k].dual_value)
#print(np.allclose(W.value,np.matrix(W.value).H))
#print(W.value)
print('Checkeo: W rango 1:')
print('-------------------')


print('Valores propios de W:')
vapW, eigW = np.linalg.eig(W.value)
print('vapw',vapW)
#print('eigw',eigW)



----------------------------------------------------------------------------
	SCS v2.1.0 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
----------------------------------------------------------------------------
Lin-sys: sparse-direct, nnz in A = 288
eps = 1.00e-04, alpha = 1.50, max_iters = 5000, normalize = 1, scale = 1.00
acceleration_lookback = 10, rho_x = 1.00e-03
Variables n = 105, constraints m = 135
Cones:	primal zero / dual free vars: 65
	linear vars: 15
	sd vars: 55, sd blks: 1
Setup time: 7.37e-03s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 1.32e+20  4.65e+19  1.00e+00 -8.91e+21  5.36e+20  8.19e+21  1.09e-02 
   100| 8.40e-06  1.01e-05  2.32e-06  1.48e+02  1.48e+02  4.03e-14  2.55e-02 
--------------------------------------------------------------------

In [25]:
voltaje=eigW[:,0]*np.sqrt(vapW[0])
print('voltaje:')


#np.linalg.trace(W.value-X.value-1j*Y.value)
print(W.value)
print(X.value)
print(abs(voltaje)*abs(voltaje))


voltaje:
[[1.10003658+0.j         1.09642196+0.08918243j 1.09624487+0.09103729j
  1.09818706+0.06372407j 1.09984492-0.02030122j]
 [1.09642196-0.08918243j 1.10004263+0.j         1.10002293+0.00185691j
  1.09974326-0.02552252j 1.09458039-0.10940995j]
 [1.09624487-0.09103729j 1.10002293-0.00185691j 1.10001161+0.j
  1.09967351-0.02737405j 1.09436912-0.11125511j]
 [1.09818706-0.06372407j 1.09974326+0.02552252j 1.09967351+0.02737405j
  1.10002171+0.j         1.0968154 -0.08398208j]
 [1.09984492+0.02030122j 1.09458039+0.10940995j 1.09436912+0.11125511j
  1.0968154 +0.08398208j 1.10002199+0.j        ]]
[[1.10002846 1.09642188 1.09624494 1.09818708 1.09984497]
 [1.09642188 1.10002727 1.10002225 1.09974328 1.09458043]
 [1.09624494 1.10002225 1.10003412 1.09967339 1.09436919]
 [1.09818708 1.09974328 1.09967339 1.10000443 1.0968154 ]
 [1.09984497 1.09458043 1.09436919 1.0968154  1.10001575]]
[1.10003835 1.10004624 1.10000388 1.10002669 1.10002242]


a) Obtener los multiplicadores de Lagrange de las distintas restricciones e interpretarlos.

b) Reflexionar sobre qué sucedería si se modificaran los costos de generación y luego correr el problema con nuevos costos.

c) Reflexionar sobre qué sucedería si se modificaran los límites de las líneas y luego correr el problema con nuevos costos.



In [0]:
#Escribir código aquí